## Языковое моделирование

Языковые модели - важнейшая часть современного NLP. Практически во всех задачах, связанных с обработкой текста, напрямую или косвенно используются языковые модели. А наиболее известные недавние прорывы в области - это по большей части новые подходы к языковому моделированию. ELMO, BERT, GPT - это языковые модели.

Это достаточно сложная тема, поэтому будем разбирать постепенно. Сегодня разберём самые основы. Научимся приписывать вероятность последовательности слов и попробуем генерировать текст.

In [72]:
# !pip install razdel

In [73]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 

Возьмем два разных корпуса: новостной и сообщения с 2ch.

In [74]:
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные
dvach = open('2ch_corpus.txt', encoding='UTF8').read()
news = open('lenta.txt', encoding='UTF8').read()

По длине они сопоставимы.

In [75]:
print("Длина 1 -", len(dvach))
print("Длина 2 -", len(news))

Длина 1 - 11638405
Длина 2 - 11536552


Напишем простую функцию для нормализации. Удалять пунктуацию и приводить к нижнему регистру, строго говоря не стоит, сгенерированный текст так будет не похож на настоящий. Но это немного упростит нам работу. 

In [76]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


Сравним тексты по токенам

In [77]:
norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [78]:
print("Длина корпуса токсичных постов в токенах -", len(norm_dvach))
print("Длина корпуса новостных текстов в токенах - ", len(norm_news))

Длина корпуса токсичных постов в токенах - 1858941
Длина корпуса новостных текстов в токенах -  1505789


И по уникальным токенам

In [79]:
print("Уникальных токенов в корпусе токсичных постов -", len(set(norm_dvach)))
print("Уникальный токенов в корпусе новостных текстов - ", len(set(norm_news)))

Уникальных токенов в корпусе токсичных постов - 189515
Уникальный токенов в корпусе новостных текстов -  116302


Посчитаем, сколько раз встречаются слова и выведем самые частотные.

In [80]:
from collections import Counter

In [81]:
vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)


In [82]:
vocab_dvach.most_common(10)

[('и', 55892),
 ('в', 48853),
 ('не', 46602),
 ('на', 29660),
 ('что', 26668),
 ('я', 21734),
 ('а', 21310),
 ('с', 21080),
 ('это', 17727),
 ('ты', 15469)]

In [83]:
vocab_news.most_common(10)

[('в', 72412),
 ('и', 33290),
 ('на', 28434),
 ('по', 19490),
 ('что', 17031),
 ('с', 15921),
 ('не', 12702),
 ('из', 7727),
 ('о', 7515),
 ('как', 7514)]

Для того, чтобы превратить абсолютные частоты в вероятности, разделим на общее число слов в каждом корпусе.

In [84]:
vocab_news['ваываываываываыва']

0

In [85]:
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})
probas_dvach.most_common(20)

[('и', 0.030066580918921042),
 ('в', 0.02628001641794979),
 ('не', 0.02506911192985684),
 ('на', 0.015955320798239428),
 ('что', 0.014345802260534357),
 ('я', 0.011691602907246653),
 ('а', 0.011463516055646737),
 ('с', 0.011339789697467536),
 ('это', 0.009536074571489897),
 ('ты', 0.008321404498582796),
 ('как', 0.007882444897390503),
 ('у', 0.006848522895562581),
 ('но', 0.005786090037284669),
 ('так', 0.005383172462170666),
 ('по', 0.005060945990217011),
 ('то', 0.005049649235774562),
 ('все', 0.0046537248896011225),
 ('за', 0.004583792600195488),
 ('же', 0.004228751746289958),
 ('если', 0.004209385881531474)]

In [86]:
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(20)

[('в', 0.04808907489694771),
 ('и', 0.0221080111489724),
 ('на', 0.018883123731146926),
 ('по', 0.012943380513471676),
 ('что', 0.011310349590812525),
 ('с', 0.01057319451795703),
 ('не', 0.008435444806676101),
 ('из', 0.005131529052211166),
 ('о', 0.00499073907433246),
 ('как', 0.0049900749706632205),
 ('к', 0.00407161959610543),
 ('за', 0.0040125143695431435),
 ('россии', 0.0036751497055696383),
 ('для', 0.003325831175549828),
 ('его', 0.003260084912295149),
 ('он', 0.0031704309169478593),
 ('от', 0.003066830744546547),
 ('сообщает', 0.003050228152815567),
 ('а', 0.0029180715226369697),
 ('также', 0.002716184007188258)]

Эти вероятности уже можно использовать, чтобы ответить на вопрос - это предложение больше подходит для новостей или для анонимного форума?

В теории вероятностей для того, чтобы найти общую вероятность нескольких независимых событий произойти одновременно, нужно перемножить вероятности отдельных событий. В нашем случае мы хотим найти вероятность получить данное предложение. Для этого мы перемножаем вероятности слов в этом предложении. 

(Если бы мы сложили вероятности, то мы бы получили вероятность выбрать из корпуса 1 из слов в данном предложении)

Напишем простую функцию, которая расчитает обшую вероятность. В прошлом семинаре мы говорили о том, что вместо умножения вероятностей можно складывать логарифмы от них. Еще нам нужно учесть одну деталь - некоторых слов может не быть в словаре и, соответственно, вероятность будет нулевая. Можно использовать в таких случаях небольшое значение вероятности, например 1/длина корпуса. Исправить это по-нормальному - сложно, придется подробнее разбираться с вероятностями, сглаживаниями и заменой неизвестных слов.

In [87]:
def compute_joint_proba(text, word_probas):
    prob = 0
    for word in normalize(text):
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += (np.log(1/len(norm_dvach)))
    
    return np.exp(prob)

In [88]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'

Расчитаем вероятность встретить такой текст в каждом из корпусов (для таких маленьких чисел нужно смотреть на степень после e: чем больше степень, тем больше вероятность; но тут легко запутаться так как степень будет отрицательная и больше будет число, которое ближе к нулю (-5 больше -10 например)

In [89]:
compute_joint_proba(phrase, probas_dvach)

3.8958314050721132e-50

In [90]:
compute_joint_proba(phrase, probas_news)

4.573351371331133e-45

Можно просто доверится функции больше/меньше, чтобы не запутаться

In [91]:
compute_joint_proba(phrase, probas_news) > compute_joint_proba(phrase, probas_dvach)

True

Вероятность встретить такой текст в новостном корпусе выше. Попробуем другой текст:

In [92]:
phrase = 'Безграмотное быдло с дубляжом, войсовером, порнографией и котикам'
compute_joint_proba(phrase, probas_news) > compute_joint_proba(phrase, probas_dvach)

False

Тут получается обратная ситуация.

Однако такая оценка вероятности предложения черезчур упрощает действительность. Слова в предложении - это не независимые события, выбор первого слова сильно влияет на вероятности выбрать второе, третье и так далее.

Такие события можно оценивать по формуле полной вероятности:

In [93]:
Image(url="https://i.ibb.co/sC7CKzQ/image.png",
     width=700, height=500)

А если простыми словами, то для того, чтобы получить вероятность предложения, нужно перемножить вероятность первого слова, вероятность второго слова, при условии первого, вероятность третьего при условии первого и второго, вероятность четвертого слова, при условии первого, второго и третьего и так далее до вероятности последнего слова при условии всех предшешевствующих.

Условные вероятности для слов можно также вычислить по частотностям. Вероятность слова А при условии слова Б равна отношению количества раз, которое встретились слова А и Б вместе, к количеству раз, которое встретилось слово Б. Вероятность слова В при условии А и Б равна отношению количества раз, которое встретились слова А,Б и В вместе к количеству раз, которое встретились слова А и Б.
И так далее. 

Но тут появляется проблема. Для того, чтобы расчитать полную вероятность предложения нужно, чтобы такое предложение уже встретилось в корпусе хотя бы 1 раз. Очевидно, что даже огромный корпус всего написанного текста не включает в себя все возможные тексты (тем более маленький корпус). Поэтому один из множителей в произведении будет нулевым, а значит и все произведение станет нулевым.

Для того, чтобы этого избежать можно поубавить строгости и предположить, что вероятность слова зависит только от предыдущего слова. Это предположение называется марковским (в честь математика Андрея Маркова). Такую модель еще можно назвать биграммной.

Чтобы расчитать вероятность с таким предположением, нам достаточно найти количество вхождений для каждого биграмма. А частоты отдельных слов у нас уже есть. 

In [94]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

Вероятность первого слово можно по идее считать просто как вероятность униграмма, но можно сделать небольшое добавление в нашу модель - поставить в начала каждого предложения технический токен начала предложения, а вероятность первого слова рассчитывать как вероятность биграма старт-первое слово поделить на частоту старта. Дальше мы будем генерировать текст с помощью языковой модели и это поможет нам генерировать более красивые предложения.


Для того, чтобы у нас получились честные вероятности и можно было посчитать вероятность первого слова, нам нужно добавить тэг маркирующий начало предложений \< start \>

Дальше мы попробуем сгенерировать текст, используя эти вероятности, и нам нужно будет когда-то остановится. Для этого добавим тэг окончания \< end \>

Ну и поделим все на предложения

In [95]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\robst\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [96]:
sentences_dvach = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach[:5000000])]
sentences_news = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [97]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))


unigrams_news = Counter()
bigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))


Чтобы посчитать условную вероятность мы можем поделить количество вхождений на количество вхождений первого слова. Обновим нашу функцию

In [98]:
def compute_joint_proba_markov_assumption(text, word_counts, bigram_counts):
    prob = 0
    for ngram in ngrammer(['<start>'] + normalize(phrase) + ['<end>']):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)
    
    return np.exp(prob)

In [99]:
# Эта фраза более вероятна в корпусе двача
phrase = 'Безграмотное быдло с дубляжом, войсовером, порнографией и котикам'

compute_joint_proba_markov_assumption(phrase, unigrams_dvach, bigrams_dvach) > \
compute_joint_proba_markov_assumption(phrase, unigrams_news, bigrams_news)

True

In [100]:
# Эта фраза более вероятна в корпусе новостей
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'

compute_joint_proba_markov_assumption(phrase, unigrams_dvach, bigrams_dvach) > \
compute_joint_proba_markov_assumption(phrase, unigrams_news, bigrams_news)

False

### Генерация текста

Попробуем теперь генерировать текст с помощью биграмной языковой модели. Принцип генерации очень простой - на каждом шаге мы случайно выбираем следующее слово согласно вероятностям, расчитанным по 1 предыдущему слову.

В генерации мы можем выбирать только из уже известных слов. Можно заранее расчитать все вероятности и сохранить их в матрицу. Размерность матрицы слова на слова. В каждой ячееке будет лежать вероятность получить слово б, после слова а. Слово а будет в строке, а б в колонке.

Матрицы получатся очень большими, но большинство значений будет нулевыми, поэтому можно воспользоваться разреженным форматом.

In [101]:
from scipy.sparse import lil_matrix

In [102]:
# матрица слова на слова (инициализируем нулями)
matrix_dvach = lil_matrix((len(unigrams_dvach), 
                         len(unigrams_dvach)))

# к матрице нужно обращаться по индексам
# поэтому зафиксируем порядок слов в словаре и сделаем маппинг id-слово и слово-id
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

# заполняем матрицу
for ngram in bigrams_dvach:
    word1, word2 = ngram.split()
    # на пересечение двух слов ставим вероятность встретить второе после первого
    matrix_dvach[word2id_dvach[word1], word2id_dvach[word2]] =  (bigrams_dvach[ngram]/
                                                                     unigrams_dvach[word1])

In [103]:
# то же самое для другого корпуса
matrix_news = lil_matrix((len(unigrams_news), 
                        len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}



for ngram in bigrams_news:
    word1, word2 = ngram.split()
    matrix_news[word2id_news[word1], word2id_news[word2]] =  (bigrams_news[ngram]/
                                                                     unigrams_news[word1])

Для генерации нам понадобится функция np.random.choice , которая выбирает случайный объект из заданных. Ещё в неё можно подать вероятность каждого объекта и она будет доставать по ним (не только максимальный по вероятности)

In [104]:
def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        # просто выбирать наиболее вероятное продолжение не получится
        # можете попробовать раскоментировать следующую строчку и посмотреть что получается
        # в современных языковых моделях есть специальный параметр, который
        # позволяет регулировать разнообразность/случайность генерации
        # он называется температура, чем выше температура тем ближе будет к argmax
        # чем меньше температура тем ближе к полностью рандомной генерации
        #         chosen = matrix[current_idx].argmax()
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [105]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach).replace('<end>', '\n'))

может и его исполнении когда бж были другие спермоворы это ты максималист которому просто взять себя теми чья это не такой пропуск это наглядная иллюстрация дискриминация 
 и не упоминать 
 мюридка просто орешь на трех крупнейших предприятий в dev ты меня расклад такой 
 сперва пишут 
 в час причем зачем тф 2 я знаю что они 
 одна труба на стене висит над ней 
 да и я ещё он сцука сц убейся нет 
 просто напечатали еще и то же роковая женщина шашлык пожарил анонче что из социальных сетях 
 такое « святое в кавычках 
 какого


In [106]:
print(generate(matrix_news, id2word_news, word2id_news).replace('<end>', '\n'))

впрочем эта дата подписания в корпорации 
 основополагающим при необходимости закон о которых обучают в городской избирательной кампании и сбиты с председателем ассоциации по утверждению минобороны поскольку если террористов 
 такое решение судьи капитолины малышевой отклонил все прошли 11791 человек считаются крупнейшим падением спроса на себя также поддерживая ее с ужесточением борьбы с помощью планировалось сокращение выбросов в вашингтоне сообщает cnn в регистрации избирательных участках 
 в районе и членами опек соглашения о нарушениях в документе обобщены и дагестана делегация которая уполномочена заявить на льготное налообложение через дунай поразил бы назвали в результате этих импровизированных производств работ продолжают поиски судна


### Перплексия

Генерировать текст полностью требуется достаточно редко. И к тому же с этим более менее адекватно справляются только самые современные огромные модели. Гораздно более практичное применения языковой модели - выбрать наиболее вероятное продолжение уже введенной фразы. Вы все сталкивались с этим в своих телефонах. Если попытаться сгенировать текст только на основе предложенных слов, то получится не сильно лучше чем в текстах выше. Также языковую модель можно использовать, чтобы выбрать наиболее подходящее по контексту исправление опечатки и в этом случае совсем не важно, насколько красивые тексты она генерирует.

Но как тогда оценивать качество языковой модели? Для этого стандартно используется перплексия (на русский обычно не переводят). У перплексии есть теоретическое обоснование в теории информации и даже какая-то интерпретация, но они достаточо сложные и непонятные. На практике можно просто считать, что перплексия показывает насколько хорошо языковая модель предсказывает корпус. Чем она ниже, тем лучше.

Считается перплексия по вот такой формуле:


In [107]:
Image(url="https://i.ibb.co/Ph3sNMp/image.png",
     width=500, height=500)

Простыми словами - нам нужно расчитать вероятность текста (мы это уже научились делать выше) и возвести ее в степень (-1/N), где N это количество слов в тексте.

In [108]:
# Мы уже видели что произведение вероятностей можно заменить на экспоненту суммы логарифмов
# С возведением в степень тоже есть удобное правило - log(x^y) = y * log(x)
# можно заменить вот такую функцию (она ожидает вероятность)
# def perplexity(p, N):
#     return p**(-1/N) 


# на вот такую (результат должен совпадать)
# функция ожидает логарифм вероятности
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

Нам нужно немного изменить функцию для расчета вероятности, чтобы возвращать N

In [109]:
# функции возвращают лог (чтобы проверить с первой функцией можно добавить np.exp(prob))
def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)
    
    return prob, len(tokens)


def compute_join_proba_markov_assumption(text, word_counts, bigram_counts):
    prob = 0
    tokens = normalize(phrase)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>']):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)
    
    return prob, len(tokens)

У нас есть две функции для генераци вероятности последовательности. По сути каждая функция - это языковая модель. Первая - униграмная, а вторая биграмная.

Таким образом мы можем сравнить две языковые модели расчитанные на одном корпусе.

In [110]:
phrase = 'Безграмотное быдло с дубляжом, войсовером, порнографией и котикам'

In [111]:
perplexity(*compute_joint_proba(phrase, probas_dvach))

17738.135055632363

In [112]:
perplexity(*compute_join_proba_markov_assumption(phrase, unigrams_dvach, bigrams_dvach))

233.89095512739362

Перплексия второй (биграмной модели) сильно меньше. Значит она лучше предсказывает корпус

**Со вторым текстом, который мы рассматривали до этого такое не сработает, потому что слишком много биграммов нет в словарях. Поэтому перплексия биграммной модели будет выше. 

In [113]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'

In [114]:
perplexity(*compute_joint_proba(phrase, probas_news))

10737.11899899257

In [115]:
perplexity(*compute_join_proba_markov_assumption(phrase, unigrams_news, bigrams_news))

12287.628005974075

Но лучше оценивать качество языковой модели не на 1 тексте, а сразу на целом корпусе. Мы можем посчитать перплексию на всех предложениях и усреднить, чтобы получить общую перплексию

Униграмная модель:

In [116]:
ps = []
for sent in sent_tokenize(dvach[:5000000]):
    prob, N = compute_joint_proba(sent, probas_dvach)
    if not N:
        continue
    ps.append(perplexity(prob, N))

In [117]:
np.mean(ps)

31346.173280655195

Бигрмная модель:

In [118]:
ps = []
for sent in sent_tokenize(dvach[:5000000]):
    prob, N = compute_join_proba_markov_assumption(sent, unigrams_dvach, bigrams_dvach)
    if not N:
        continue
    ps.append(perplexity(prob, N))

In [119]:
np.mean(ps)

82659.58358943973